# Advent of Code 2021 - Day 10

In [1]:
enum class Bracket(val opening: Char, val closing: Char, val exceptionScore: Int, val successScore: Int) {
    PAREN('(', ')', 3, 1),
    SQUARE('[', ']', 57, 2),
    CURLY('{', '}', 1197, 3),
    ANGLE('<', '>', 25137, 4),
    ;

    companion object {
        private val openingMap: Map<Char, Bracket> = Bracket.values().associateBy(Bracket::opening)
        private val closingMap: Map<Char, Bracket> = Bracket.values().associateBy(Bracket::closing)

        fun fromOpening(char: Char): Bracket? = openingMap[char]
        fun fromClosing(char: Char): Bracket? = closingMap[char]
        fun from(char: Char): Bracket? = fromOpening(char) ?: fromClosing(char)
    }
}

In [ ]:
import java.io.File

val chunks: List<String> = File("Day10.input.txt")
  .bufferedReader()
  .readLines()

## Part 1

Every line of `chunks` contains zero or more other chunks. Every chunk must open and close with their respective `Bracket`. Legal chunks include: `([])`, `{()()()}`, `[<>({})[]]`, etc. Find all of the lines where a chunk closes with the wrong character. Incomplete chunks do not count. i.e. `(]`, `{()()()>`, etc. Take the first illegal character of every line and sum them by their `Bracket::exceptionScore`. What was the total score?

In [ ]:
sealed interface ParseResult

data class ParseSuccess(val stack: ArrayDeque<Bracket>) : ParseResult
data class ParseException(val expected: Char, val found: Char) : ParseResult

fun String.parse(): ParseResult {
    val stack = ArrayDeque<Bracket>()
    
    forEach {
        val opening = Bracket.fromOpening(it)
        if (opening != null) stack.addFirst(opening)
        else {
            val closing = Bracket.fromClosing(it)
            if (closing != null && closing == stack.first()) stack.removeFirst()
            else return ParseException(expected = stack.first().closing, found = closing?.closing ?: it)
        }
    }

    return ParseSuccess(stack)
}

chunks
    .map { it.parse() }
    .filterIsInstance<ParseException>()
    .mapNotNull { Bracket.from(it.found) }
    .sumOf(Bracket::exceptionScore)

268845

### Notes

The problem is very closely modeled to a stack. The idea is to push opening brackets to the top of the stack, and when you encounter a closing bracket, check if the top of the stack can be closed with the current character. If it can't then it's an illegal character.

## Part 2

Complete any incomplete chunks by matching closing out any open `Bracket`s. For each line, calculate the score by the following: Start with a total score of 0, then for each added bracket, multiply the total score by 5 then increase the score by its respective `Bracket::successScore`. Sort the score and find the middle score.

In [ ]:
fun Iterable<Bracket>.score(): Long = fold(0L) { acc, it -> acc * 5 + it.successScore }

chunks
    .map { it.parse() }
    .filterIsInstance<ParseSuccess>()
    .map { it.stack.score() }
    .sorted()
    .let { it[it.size / 2] }

4038824534

### Notes

Since we've modeled the problem as a stack, any elements still remaining on the stack tells us what still need to be added in order, with this the score is just a simple fold.

I went back and modified `String::parse` from part 1 to return a `sealed interface` and was really happy with how it turned out, it originally only had `ParseException`. It meant that I could just use the original parsing logic without any changes and simply return either the exception (for part 1) or the state of the stack (part 2).